## Data Cleaning

In [11]:
import pandas as pd 
import numpy as np 
from pymongo import MongoClient
import json

### Database Connection

In [1]:
DB_USERNAME = 'anutibara'
DB_PASSWORD = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'
try:
    client = MongoClient(f'mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except:
    print("Error to connect to database")
db = client.get_database('scraping_db')

NameError: name 'client' is not defined

### Database Queries

In [13]:
properties = db.properties
properties.count_documents({})

29468